65


/tmp/ipykernel_720/834705454.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.best_loss_model_path, map_location=args.device))  # 가

In [ ]:

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn as nn
import torch.nn.functional as F

from model_att import Seq2Seq
from dataset2 import NMT_Dataset, prepare_data, tensor_from_pair

from tqdm import tqdm




import argparse

parser = argparse.ArgumentParser()
args = parser.parse_args("")

## Device ##
args.device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

## Eval ##
args.criterion = torch.nn.CrossEntropyLoss(ignore_index=2)  # Optional ignore padding



## Data sttting parameter ##
args.is_shuffle = True

## Gradient/Optimizer ##
args.max_grad_norm = 5
args.learning_rate_gamma = 0.5

## Path ##
args.best_loss_model_path = 'model_local/local_model.pth'  # model_new :처음적용한 새 전처리 # model 두번쨰 전처리


## Model ##
args.hidden_dimension = 1000
args.max_len = 9
args.min_len = 6

args.n_layers = 4
args.dropout = 0.2
args.attention_type = 'local' #'local' # 'global'
args.align_type = 'general' #'general'
args.input_feeding = True # True

## Learning ##
args.lr = 1.0
args.epoch = 10
args.batch_size = 128
args.lr_milestone = [6, 7, 8, 9, 10]
args.use_saved_vocab = False
args.local_window = 10 
## Options depend on dropout version
if args.dropout > 0.0:
    args.epoch = 12
    args.lr_milestone = [8, 9, 10, 11, 12]

## data prepareation:


input_vocab, output_vocab, test_pairs = prepare_data('test',args.max_len,args.min_len, True)


test_input_tensors, test_output_tensors,test_source_len_tensors = tensor_from_pair(test_pairs, input_vocab, output_vocab,args.max_len, "test")
print(len(test_pairs))

#test_input_tensors = test_input_tensors.flip(1)

test_nmt_dataset = NMT_Dataset(test_input_tensors, test_output_tensors,test_source_len_tensors)
test_dataloader = DataLoader(test_nmt_dataset, batch_size = args.batch_size, shuffle = False, drop_last=False)


# Set random seed for reproducibility
import random
import numpy as np

seed = 42  # 원하는 시드 값으로 변경 가능
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

## Model
model = Seq2Seq(input_vocab.n_words, output_vocab.n_words ,args.hidden_dimension, args.hidden_dimension, args.batch_size, 
                args.device, args.n_layers, args.dropout, args.max_len,args.attention_type,args.align_type,args.input_feeding,args.local_window)
model.to(args.device)

model.load_state_dict(torch.load(args.best_loss_model_path, map_location=args.device))  # 가중치 로드

## Training
train_losses = []
dev_losses = []


train_ppls = []
dev_ppls = []
# 파일 경로 지정
output_file = "gen_sentences.txt"
ref_file = "ref_sentences.txt"

# 파일 초기화 (기존 파일 삭제 후 새로 생성)
with open(output_file, "w", encoding="utf-8") as f1, open(ref_file, "w", encoding="utf-8") as f2:

    model.eval()


    for i, (input_tensor, output_tensor,source_len_tensor) in enumerate(zip(test_input_tensors, test_output_tensors,test_source_len_tensors)):

        input_tensor = input_tensor.to(args.device)
        output_tensor = output_tensor.to(args.device)
        source_len_tensor = source_len_tensor.to(args.device)



        ## view 연산은 메모리 공간상에 연속적인 공간을 가정한다.
        ## 하지만, padding 연산 같은 경우에는 메모리 공간상의 연속성을 보장하지 못하므로
        ## decoder_target에는 .contiguous 연산을 적용해줌
        ## decoder_outputs 같은 경우에는 Torch Tensor 연산의 결과물이므로
        ## 게산과정에서 연속 공간임이 보장됨

        # print(decoder_outputs.size()) # torch.Size([2, 51, 1325509])
        # print(decoder_target.size()) # torch.Size([2, 51])

        encoder_outputs, encoder_hidden, encoder_cell = model.encoder(input_tensor.unsqueeze(0))
        
        # 디코더의 첫 입력은 <SOS> 토큰
        decoder_input = torch.tensor([[output_vocab.word2index['SOS']]]).to(args.device)
                
        attention_output = torch.zeros(1, 1, args.hidden_dimension, device=args.device)
        
        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell

        # 빔 서치를 위한 초기 설정: 시퀀스, 확률, 히든 상태를 담는 리스트
        sequences = [[[], 1.0, decoder_hidden, decoder_cell,attention_output]]
        
        #print(decoder_input.shape)
        # 디코더의 첫 입력을 배치 크기 128로 초기화
        # decoder_input = torch.full((128, 1), output_vocab.word2index['SOS'], dtype=torch.long).to(args.device)        
        
        
        # print(decoder_hidden.shape) #torch.Size([4, 1, 1000])
        
        #################################
        for i in range(args.max_len+1):
            
            all_candidates = []
            
            for seq, score, decoder_hidden, decoder_cell,attention_output in sequences:
                # <EOS>가 이미 예측된 경우 해당 시퀀스는 확장하지 않고 유지
                if len(seq) > 0 and seq[-1] == output_vocab.word2index['EOS']:
                    all_candidates.append([seq, score, decoder_hidden, decoder_cell,attention_output])
                    continue 
                
                last_word = torch.tensor([[seq[-1] if seq else output_vocab.word2index['SOS']]]).to(args.device)
                decoder_output, decoder_hidden , decoder_cell, attention_output,align_fn = model.decoder.forward_step(encoder_outputs, last_word, decoder_hidden, decoder_cell,attention_output,source_len_tensor)

                decoder_output = torch.nn.functional.log_softmax(decoder_output, dim=-1)
                # 상위 beam_width개의 단어 선택
                beam_width = 5
                topk_probs, topk_indices = torch.topk(decoder_output, beam_width, dim=-1)
                
                for j in range(beam_width):
                    word_idx = topk_indices[0, 0, j].item()  # 상위 i번째 단어 인덱스
                    prob = topk_probs[0, 0, j].item()         # 해당 단어의 확률
                    
                    # 새로운 후보 시퀀스를 확장
                    candidate = [seq + [word_idx], score + prob, decoder_hidden, decoder_cell,attention_output]
                    all_candidates.append(candidate)

            # 확장된 후보 중 상위 beam_width개의 시퀀스만 유지
            ordered = sorted(all_candidates, key=lambda x: x[1], reverse=True)
            sequences = ordered[:beam_width]
            
            # 모든 시퀀스가 <EOS>로 끝나는 경우 조기 종료
            if all(seq[-1] == output_vocab.word2index['EOS'] for seq, _, _, _,_ in sequences):
                break
                

        # # 가장 높은 점수를 가진 시퀀스를 반환
        best_sequence = sequences[0][0]
        ########################################
        # generated_sequence =[]
        # for _ in range(args.max_len):
        #     # 디코더에서 한 단계 예측 수행
        #     decoder_output, decoder_hidden, decoder_cell = model.decoder.forward_step(
        #         decoder_input, decoder_hidden, decoder_cell
        #     )
            
        #     # 가장 높은 확률의 단어 선택
        #     top_word = torch.argmax(decoder_output, dim=-1).item()
        #     generated_sequence.append(top_word)
            
        #     # <EOS> 토큰이 예측되면 종료
        #     if top_word == output_vocab.word2index['EOS']:
        #         break
            
        #     # 다음 입력을 현재 예측된 단어로 설정
        #     decoder_input = torch.tensor([[top_word]]).to(args.device)

        # # 시퀀스를 문자열로 변환
        # best_sequence = " ".join([output_vocab.index2word[word] for word in generated_sequence])
        #print(best_sequence)
        
        
        
        best_sequence = best_sequence[:-1]
        sen = ""
        for seq in best_sequence:
            w = output_vocab.index2word[seq]
            sen +=  w +" "


        sen2 = ""
        for ot in output_tensor:
            if ot.item() ==1:
                break
            w = output_vocab.index2word[ot.item()]
            sen2 += w + " "

        f1.write(sen.strip()+"\n")
        f2.write(sen2.strip()+"\n")

                                     

    

185


/tmp/ipykernel_3532/1135750457.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.best_loss_model_path, map_location=args.device))  #

In [4]:

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn as nn
import torch.nn.functional as F

from model_att import Seq2Seq
from dataset2 import NMT_Dataset, prepare_data, tensor_from_pair

from tqdm import tqdm




import argparse

parser = argparse.ArgumentParser()
args = parser.parse_args("")

## Device ##
args.device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

## Eval ##
args.criterion = torch.nn.CrossEntropyLoss(ignore_index=2)  # Optional ignore padding



## Data sttting parameter ##
args.is_shuffle = True

## Gradient/Optimizer ##
args.max_grad_norm = 5
args.learning_rate_gamma = 0.5

## Path ##
args.best_loss_model_path = 'model_local/local_model.pth'  # model_new :처음적용한 새 전처리 # model 두번쨰 전처리


## Model ##
args.hidden_dimension = 1000
args.max_len = 80
args.min_len = 45
args.n_layers = 4
args.dropout = 0.2
args.attention_type = 'local' # 'global'
args.align_type = 'general'
args.input_feeding = True # True

## Learning ##
args.lr = 1.0
args.epoch = 10
args.batch_size = 128
args.lr_milestone = [6, 7, 8, 9, 10]
args.use_saved_vocab = False
args.local_window = 10 
## Options depend on dropout version
if args.dropout > 0.0:
    args.epoch = 12
    args.lr_milestone = [8, 9, 10, 11, 12]

## data prepareation:


input_vocab, output_vocab, test_pairs = prepare_data('test',args.max_len,args.min_len, True)


test_input_tensors, test_output_tensors,test_source_len_tensors = tensor_from_pair(test_pairs, input_vocab, output_vocab,args.max_len, "test")
print(len(test_pairs))

test_input_tensors = test_input_tensors.flip(1)

test_nmt_dataset = NMT_Dataset(test_input_tensors, test_output_tensors,test_source_len_tensors)
test_dataloader = DataLoader(test_nmt_dataset, batch_size = args.batch_size, shuffle = False, drop_last=False)

#test_input_tensors = test_input_tensors.flip(1)
# Set random seed for reproducibility
import random
import numpy as np

seed = 42  # 원하는 시드 값으로 변경 가능
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

## Model
model = Seq2Seq(input_vocab.n_words, output_vocab.n_words ,args.hidden_dimension, args.hidden_dimension, args.batch_size, 
                args.device, args.n_layers, args.dropout, args.max_len,args.attention_type,args.align_type,args.input_feeding,args.local_window)
model.to(args.device)

model.load_state_dict(torch.load(args.best_loss_model_path, map_location=args.device))  # 가중치 로드

## Training
train_losses = []
dev_losses = []


train_ppls = []
dev_ppls = []
# 파일 경로 지정
output_file = "gen_sentences.txt"
ref_file = "ref_sentences.txt"

# 파일 초기화 (기존 파일 삭제 후 새로 생성)
with open(output_file, "w", encoding="utf-8") as f1, open(ref_file, "w", encoding="utf-8") as f2:

    model.eval()


    for i, (input_tensor, output_tensor,source_len_tensor) in enumerate(zip(test_input_tensors, test_output_tensors,test_source_len_tensors)):

        input_tensor = input_tensor.to(args.device)
        output_tensor = output_tensor.to(args.device)
        source_len_tensor = source_len_tensor.to(args.device)



        ## view 연산은 메모리 공간상에 연속적인 공간을 가정한다.
        ## 하지만, padding 연산 같은 경우에는 메모리 공간상의 연속성을 보장하지 못하므로
        ## decoder_target에는 .contiguous 연산을 적용해줌
        ## decoder_outputs 같은 경우에는 Torch Tensor 연산의 결과물이므로
        ## 게산과정에서 연속 공간임이 보장됨

        # print(decoder_outputs.size()) # torch.Size([2, 51, 1325509])
        # print(decoder_target.size()) # torch.Size([2, 51])

        encoder_outputs, encoder_hidden, encoder_cell = model.encoder(input_tensor.unsqueeze(0))
        
        # 디코더의 첫 입력은 <SOS> 토큰
        decoder_input = torch.tensor([[output_vocab.word2index['SOS']]]).to(args.device)
                
        attention_output = torch.zeros(1, 1, args.hidden_dimension, device=args.device)
        
        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell

        # 빔 서치를 위한 초기 설정: 시퀀스, 확률, 히든 상태를 담는 리스트
        sequences = [[[], 1.0, decoder_hidden, decoder_cell,attention_output]]
        
        #print(decoder_input.shape)
        # 디코더의 첫 입력을 배치 크기 128로 초기화
        # decoder_input = torch.full((128, 1), output_vocab.word2index['SOS'], dtype=torch.long).to(args.device)        
        
        
        # print(decoder_hidden.shape) #torch.Size([4, 1, 1000])
        
        #################################
        for i in range(args.max_len+1):
            
            all_candidates = []
            
            for seq, score, decoder_hidden, decoder_cell,attention_output in sequences:
                # <EOS>가 이미 예측된 경우 해당 시퀀스는 확장하지 않고 유지
                if len(seq) > 0 and seq[-1] == output_vocab.word2index['EOS']:
                    all_candidates.append([seq, score, decoder_hidden, decoder_cell,attention_output])
                    continue 
                
                last_word = torch.tensor([[seq[-1] if seq else output_vocab.word2index['SOS']]]).to(args.device)
                decoder_output, decoder_hidden , decoder_cell, attention_output,align_fn = model.decoder.forward_step(encoder_outputs, last_word, decoder_hidden, decoder_cell,attention_output,source_len_tensor)

                decoder_output = torch.nn.functional.log_softmax(decoder_output, dim=-1)
                # 상위 beam_width개의 단어 선택
                beam_width = 5
                topk_probs, topk_indices = torch.topk(decoder_output, beam_width, dim=-1)
                
                for j in range(beam_width):
                    word_idx = topk_indices[0, 0, j].item()  # 상위 i번째 단어 인덱스
                    prob = topk_probs[0, 0, j].item()         # 해당 단어의 확률
                    
                    # 새로운 후보 시퀀스를 확장
                    candidate = [seq + [word_idx], score + prob, decoder_hidden, decoder_cell,attention_output]
                    all_candidates.append(candidate)

            # 확장된 후보 중 상위 beam_width개의 시퀀스만 유지
            ordered = sorted(all_candidates, key=lambda x: x[1], reverse=True)
            sequences = ordered[:beam_width]
            
            # 모든 시퀀스가 <EOS>로 끝나는 경우 조기 종료
            if all(seq[-1] == output_vocab.word2index['EOS'] for seq, _, _, _,_ in sequences):
                break
                

        # # 가장 높은 점수를 가진 시퀀스를 반환
        best_sequence = sequences[0][0]
        
        
        best_sequence = best_sequence[:-1]
        sen = ""
        for seq in best_sequence:
            w = output_vocab.index2word[seq]
            sen +=  w +" "


        sen2 = ""
        for ot in output_tensor:
            if ot.item() ==1:
                break
            w = output_vocab.index2word[ot.item()]
            sen2 += w + " "

        f1.write(sen.strip()+"\n")
        f2.write(sen2.strip()+"\n")

                                     

    

78


/tmp/ipykernel_9941/4245917592.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.best_loss_model_path, map_location=args.device))  #

In [20]:
output_tensor

tensor([ 3663,   154,    51, 16646,    16,     3,  7658,     4,    46,    79,
         2477,  3492,  3880,    71,   439,  2747,     5,     1,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2], device='cuda:1')

In [5]:


model.eval()

test_losses = []
test_ppls = []

test_loss = 0.0
test_ppl = 0.0

with torch.no_grad():
    for j, (test_input_tensors, test_output_tensor,source_len_tensor) in enumerate(test_dataloader):
        

        test_input_tensors = test_input_tensors.to(args.device)
        test_output_tensor = test_output_tensor.to(args.device)
        source_len_tensor = source_len_tensor.to(args.device)
        
        #print(test_input_tensors.size())
        test_input_tensors = test_input_tensors.flip(1)
        
        
        test_decoder_outputs, test_decoder_hidden = model(test_input_tensors,test_output_tensor,source_len_tensor)

        test_decoder_target = test_output_tensor

        tloss = args.criterion(test_decoder_outputs.view(-1, test_decoder_outputs.size(-1)), test_decoder_target.contiguous().view(-1))
        test_loss += tloss.item()
        # Perplexity 계산
        t_perplexity = torch.exp(tloss)
        test_ppl += t_perplexity.item()
        
        
    test_loss = test_loss/len(test_dataloader)
    test_ppl = test_ppl/len(test_dataloader)
                    
    test_losses.append(test_loss)
    test_ppls.append(test_ppl)
    
    print(f"This is the test loss : {test_loss} ")
    print(f"This is the test ppl : {test_ppl} ")

This is the test loss : 4.779623985290527 
This is the test ppl : 119.0595703125 


In [2]:
import torch
aa = torch.arange(50).unsqueeze(0).expand(128, -1) #128,50

In [4]:
pt = [6.1481,  7.6745,  5.1226,  4.1096, 20.4836, 14.2694, 14.3112, 14.8627,                                                                                              
    11.2228, 10.7724,  6.1617,  8.2051, 22.5600,  6.1621, 14.3348, 16.9657,                                                                                                         
    12.7363, 11.7934, 10.7760,  9.7463,  5.6555,  6.1353,  7.6987,  5.6327,                                                                                                         
    6.1402,  8.2182,  4.5978,  6.6653,  6.6677, 17.4256,  6.1673, 12.2781,                                                                                                         
    5.6417,  5.1286,  8.6765,  5.6031, 10.7644,  5.1158, 11.7404,  8.7012,                                                                                                         
    14.3956,  7.1811,  5.6543, 15.8792, 10.2079,  5.6362,  8.7269, 10.2492,                                                                                                         
    15.3738, 16.3138,  6.1440,  4.6143,  5.6339,  6.6598,  3.5841, 11.7391,                                                                                                         
    8.2086,  4.6102,  8.7010, 11.7641, 18.3843, 18.9755,  8.6957,  6.6070,                                                                                                         
    7.1694,  4.1006, 15.8979,  9.7518, 10.7843, 10.7042,  8.6585,  5.1262,                                                                                                         
    18.4144,  8.2072, 12.7400, 12.2714, 17.9672,  3.5883,  3.5798, 11.7952,                                                                                                         
    10.7170,  6.6358, 13.2956,  9.1943,  8.1645,  9.7411, 10.7111,  6.6592,                                                                                                         
    8.1967,  9.2297, 16.3910, 15.8913,  9.2178,  5.6291, 10.7522,  7.6771,                                                                                                         
    18.9118, 22.6130, 13.3018, 13.3109,  5.1144,  5.6517,  7.1645, 15.9232,                                                                                                         
    11.2854, 14.8076, 13.2719,  7.1621, 16.8708, 22.0817, 18.4360,  8.6998,                                                                                                         
    5.1246, 15.3802, 12.2392,  6.1423,  7.6669,  6.6530,  9.7128,  6.1608,                                                                                                         
    14.9250,  6.1331, 20.5066, 10.7287,  7.1994,  5.1333,  6.6567, 10.7687]

pt_t = 50 - torch.tensor(pt)

print(pt_t)

tensor([43.8519, 42.3255, 44.8774, 45.8904, 29.5164, 35.7306, 35.6888, 35.1373,
        38.7772, 39.2276, 43.8383, 41.7949, 27.4400, 43.8379, 35.6652, 33.0343,
        37.2637, 38.2066, 39.2240, 40.2537, 44.3445, 43.8647, 42.3013, 44.3673,
        43.8598, 41.7818, 45.4022, 43.3347, 43.3323, 32.5744, 43.8327, 37.7219,
        44.3583, 44.8714, 41.3235, 44.3969, 39.2356, 44.8842, 38.2596, 41.2988,
        35.6044, 42.8189, 44.3457, 34.1208, 39.7921, 44.3638, 41.2731, 39.7508,
        34.6262, 33.6862, 43.8560, 45.3857, 44.3661, 43.3402, 46.4159, 38.2609,
        41.7914, 45.3898, 41.2990, 38.2359, 31.6157, 31.0245, 41.3043, 43.3930,
        42.8306, 45.8994, 34.1021, 40.2482, 39.2157, 39.2958, 41.3415, 44.8738,
        31.5856, 41.7928, 37.2600, 37.7286, 32.0328, 46.4117, 46.4202, 38.2048,
        39.2830, 43.3642, 36.7044, 40.8057, 41.8355, 40.2589, 39.2889, 43.3408,
        41.8033, 40.7703, 33.6090, 34.1087, 40.7822, 44.3709, 39.2478, 42.3229,
        31.0882, 27.3870, 36.6982, 36.68

In [27]:
diff_ = aa-pt_t
print(diff_) 
print(torch.exp(-(diff_**2)/50))

tensor([[ -6.1481,  -5.1481,  -4.1481,  ...,  40.8519,  41.8519,  42.8519],
        [ -7.6745,  -6.6745,  -5.6745,  ...,  39.3255,  40.3255,  41.3255],
        [ -5.1226,  -4.1226,  -3.1226,  ...,  41.8774,  42.8774,  43.8774],
        ...,
        [ -5.1333,  -4.1333,  -3.1333,  ...,  41.8667,  42.8667,  43.8667],
        [ -6.6567,  -5.6567,  -4.6567,  ...,  40.3433,  41.3433,  42.3433],
        [-10.7687,  -9.7687,  -8.7687,  ...,  36.2313,  37.2313,  38.2313]])
tensor([[4.6955e-01, 5.8857e-01, 7.0883e-01,  ..., 3.1938e-15, 6.1088e-16,
         1.1226e-16],
        [3.0791e-01, 4.1025e-01, 5.2519e-01,  ..., 3.6925e-14, 7.5072e-15,
         1.4664e-15],
        [5.9166e-01, 7.1183e-01, 8.2282e-01,  ..., 5.8535e-16, 1.0746e-16,
         1.8954e-17],
        ...,
        [5.9036e-01, 7.1057e-01, 8.2172e-01,  ..., 5.9593e-16, 1.0945e-16,
         1.9313e-17],
        [4.1221e-01, 5.2731e-01, 6.4811e-01,  ..., 7.2946e-15, 1.4239e-15,
         2.6705e-16],
        [9.8343e-02, 1.4830e-01,

In [ ]:
seq_range = torch.arange(50).unsqueeze(0).expand(128, -1)

left = torch.tensor([ 0.,  3.,  1.,  0.,  1.,  0.,  5.,  6.,  6.,  8.,  0., 12.,  0.,  0.,                                                                                                           
         0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,                                                                                                           
         0.,  0.,  0.,  4.,  0.,  0., 11.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,                                                                                                           
         0.,  0.,  0.,  0.,  0.,  7.,  2.,  6.,  0.,  0.,  7.,  0.,  0.,  0.,                                                                                                           
         0.,  1.,  8.,  0.,  0.,  3., 15.,  0.,  9.,  1.,  0.,  0.,  0., 10.,                                                                                                           
        10.,  0.,  0.,  4.,  1.,  0.,  0.,  0.,  0.,  5., 13.,  6.,  0.,  0.,                                                                                                           
         0.,  0.,  2.,  0.,  0., 13.,  8.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,                                                                                                           
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  6.,  6.,  0.,  5., 11.,  0.,                                                                                                           
         1.,  5.,  4.,  0.,  0.,  9.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,  2.,                                                                                                           
         0.,  0.])                                                                                                                        
right = torch.tensor([18., 23., 21., 15., 21., 16., 25., 26., 26., 28., 15., 32., 17., 17.,                                                                                                           
        19., 19., 18., 14., 18., 18., 27., 19., 16., 20., 25., 17., 19., 18.,                                                                                                           
        15., 19., 19., 24., 20., 15., 31., 19., 14., 17., 16., 17., 21., 16.,                                                                                                           
        20., 17., 18., 18., 19., 27., 22., 26., 19., 15., 27., 17., 20., 15.,                                                                                                           
        19., 21., 28., 20., 17., 23., 35., 16., 29., 21., 15., 16., 16., 30.,                                                                                                           
        30., 16., 16., 24., 21., 13., 14., 19., 15., 25., 33., 26., 17., 16.,                                                                                                           
        16., 15., 22., 14., 17., 33., 28., 18., 18., 20., 20., 15., 23., 18.,                                                                                                           
        14., 20., 19., 18., 20., 17., 19., 22., 26., 26., 18., 25., 31., 15.,                                                                                                           
        21., 25., 24., 14., 17., 29., 16., 16., 15., 23., 16., 15., 17., 22.,                                                                                                           
        18., 16.])